# Parsing ImpactT.in

Here are some examples of the low level routines to parse the main input file.

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
from impact.parsers import *

In [3]:
from impact.lattice import *

In [4]:
with open('../templates/lcls_injector/ImpactT.in', 'r') as f:    
    data = f.read()
    LINES = data.split('\n')

In [5]:
parse_header(LINES)

{'Npcol': 1,
 'Nprow': 1,
 'Dt': 5e-13,
 'Ntstep': 1000000,
 'Nbunch': 1,
 'Dim': 6,
 'Np': 300004,
 'Flagmap': 1,
 'Flagerr': 0,
 'Flagdiag': 2,
 'Flagimg': 1,
 'Zimage': 0.02,
 'Nx': 64,
 'Ny': 64,
 'Nz': 128,
 'Flagbc': 1,
 'Xrad': 0.015,
 'Yrad': 0.015,
 'Perdlen': 45.0,
 'Flagdist': 16,
 'Rstartflg': 0,
 'Flagsbstp': 0,
 'Nemission': 400,
 'Temission': 1.4e-11,
 'sigx(m)': 0.0006,
 'sigpx': 0.0,
 'muxpx': 0.0,
 'xscale': 1.0,
 'pxscale': 1.0,
 'xmu1(m)': 0.0,
 'xmu2': 0.0,
 'sigy(m)': 0.0006,
 'sigpy': 0.0,
 'muxpy': 0.0,
 'yscale': 1.0,
 'pyscale': 1.0,
 'ymu1(m)': 0.0,
 'ymu2': 0.0,
 'sigz(m)': 1.27e-06,
 'sigpz': 0.0,
 'muxpz': 0.0,
 'zscale': 1.0,
 'pzscale': 1.0,
 'zmu1(m)': 0.0,
 'zmu2': 0.0,
 'Bcurr': 0.714,
 'Bkenergy': 1.0,
 'Bmass': 511005.0,
 'Bcharge': -1.0,
 'Bfreq': 2856000000.0,
 'Tini': 0.0}

In [6]:
# Find index of the line where the lattice starts
IX_LATTICE = ix_lattice(LINES)

# Gather lattice lines
LATLINES = LINES[IX_LATTICE:]

In [7]:
# Summary of elements in this lattice
def myf(t): return [l for l in LATLINES if parse_type(l) == t ] 
quads = myf('quadrupole')
drifts = myf('drift')
solrfs = myf('solrf')
wakefields = myf('wakefield')

alltypes = set([parse_type(l) for l in LATLINES])
print(alltypes)

for t in ele_type.values():
    l = len(myf(t))
    if l > 0:
        print(l, t)

{'comment', 'solrf', 'stop', 'drift', 'write_beam', 'wakefield', 'change_timestep', 'quadrupole', 'rotationally_symmetric_to_3d'}
31 drift
7 quadrupole
10 solrf
5 write_beam
1 change_timestep
1 rotationally_symmetric_to_3d
2 wakefield
1 stop


# Individual element parsing

In [8]:
parse_drift(drifts[0])

{'zedge': 1.387, 'radius': 0.15}

In [9]:
parse_quadrupole(quads[0])

{'zedge': 4.752856,
 'b1_gradient': 1.8524,
 'L_effective': 0.108,
 'radius': 0.016,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0}

In [10]:
parse_solrf(solrfs[0])

{'zedge': 0.0,
 'rf_field_scale': 47000000.0,
 'rf_frequency': 2856000000.0,
 'theta0_deg': 293.54,
 'filename': 'rfdata201',
 'radius': 0.15,
 'x_offset': 0.0,
 'y_offset': 0.0,
 'x_rotation': 0.0,
 'y_rotation': 0.0,
 'z_rotation': 0.0,
 'solenoid_field_scale': 0.0}

In [11]:
parse_wakefield(wakefields[0])

{'s_begin': 1.485,
 's_end': 4.527856,
 'method': 'analytical',
 'iris_radius': 0.0116,
 'gap': 0.0292,
 'period': 0.035}

# Full lattice parsing into element dicts

In [12]:
from impact.lattice import ele_str

In [13]:
# This parses all lines. 
eles = parse_lattice(LATLINES)

In [14]:
# Nice printing
for e in eles:
    s = ele_str(e)
    print(s)

change_timestep_1: change_timestep, description=, dt=5e-13, s=0.25
!**************************1.5 cell gun ************************
GUN: solrf, description=name:GUN, L=0.15, zedge=0.0, rf_field_scale=47000000.0,
      rf_frequency=2856000000.0, theta0_deg=293.54, filename=rfdata201, radius=0.15,
      x_offset=0.0, y_offset=0.0, x_rotation=0.0, y_rotation=0.0, z_rotation=0.0,
      solenoid_field_scale=0.0, s=0.15
SOL1: solrf, description=name:SOL1, L=0.49308, zedge=0.0, rf_field_scale=0.0,
      rf_frequency=0.0, theta0_deg=0.0, filename=rfdata102, radius=0.15, x_offset=0.0,
      y_offset=0.0, x_rotation=0.0, y_rotation=0.0, z_rotation=0.0,
      solenoid_field_scale=0.243, s=0.49308
WRITE_BEAM_41_BEGL0: write_beam, description=name:WRITE_BEAM_41_BEGL0,
      filename=fort.41, sample_frequency=1, s=0.977844
!!!0 0 0 -8  0 -1 0.977844    /!name:'sc_off'
drift_1: drift, description=, L=0.072, zedge=1.387, radius=0.15, s=1.459

!**************************** L0A begin *******************

In [15]:
# Output lines
for e in eles:
    print(ele_line(e))

0 0 0 -4 0.0 0.0 0.25 5e-13 /!name:change_timestep_1
!**************************1.5 cell gun ************************
0.15 0 0 105 0.0 47000000.0 2856000000.0 293.54 201 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:GUN
0.49308 0 0 105 0.0 0.0 0.0 0.0 102 0.15 0.0 0.0 0.0 0.0 0.0 0.243 /!name:SOL1
0 1 41 -2 0.0 0.0 0.977844 /!name:WRITE_BEAM_41_BEGL0
!!!0 0 0 -8  0 -1 0.977844    /!name:'sc_off'
0.072 0 0 0 1.387 0.15 /!name:drift_1
!
!**************************** L0A begin **************************
0.026 0 0 0 1.459 0.15 /!name:drift_2
0 -1 0 -6 1 1 1.485 4.527856 0.0116 0.0292 0.035 /!name:wakefield_1
!to get 63.6MeV
0.052464 0 0 105 1.485 25905612.0 2856000000.0 258.5 4 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:L0A_entrance
2.937928 0 0 105 1.537464 29913224.7 2856000000.0 288.5 5 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:L0A_body_1
2.937928 0 0 105 1.537464 29913224.7 2856000000.0 348.5 6 0.15 0.0 0.0 0.0 0.0 0.0 0.0 /!name:L0A_body_2
0.052464 0 0 105 4.475392 25905612.0 2856000000.0 258.5 7 0.15 0.0 0